# Postprocess predictions and create a submission

## Configure

In [1]:
#@title Submission notes and params { run: "auto" }
version_notes = "BBox reduction" #@param {type:"string"}

reduce_method = "weighted_boxes_fusion" #@param ["None", "nms", "soft_nms", "non_maximum_weighted", "weighted_boxes_fusion"]

iou_threshold = 0.6 #@param {type:"slider", min:0, max:1, step:0.01}
skip_box_threshold = 0.01 #@param {type:"slider", min:0, max:1, step:0.01}
sigma = 0.1 #@param {type:"slider", min:0, max:1, step:0.01}

reduce_filter = False #@param {type:"boolean"}

binary_abnormal_threshold = 0.05 #@param {type:"slider", min:0, max:1, step:0.01}
binary_normal_threshold = 0.95 #@param {type:"slider", min:0, max:1, step:0.01}
assert(binary_abnormal_threshold <= binary_normal_threshold)

SUBMIT = False #@param {type:"boolean"}

In [2]:
INPUTS_YOLO_VERSIONS = {
    5: {
        "version_notes": "YOLOv5x, 50 epochs, random rad, 20% valid split, 1024 size, IOU 0.35, conf 0.15",
        # "path": f"v{version}",
        # "enabled": version in yolo_versions_include,
    },
    6: { "version_notes": "WBF preproc, YOLOv5x, 50 epochs, 20% valid split, 1024 size, IOU 0.35, conf 0.15", },
    7: { "version_notes": "WBF preproc, YOLOv5x, 40 epochs, 20% valid split, 1024 size, IOU 0.6, conf 0.1", },
    8: { "version_notes": "WBF preproc with IoU 0.4, YOLOv5x, 40 epochs, 20% valid split, 1024 size, IOU 0.35, conf 0.15, remove (most) augs", },
    9: { "version_notes": "WBF preproc with IoU 0.7, YOLOv5x, 40 epochs, 20% valid split, 1024 size, IOU 0.6, conf 0.1, remove (most) augs", },
    10: { "version_notes": "WBF preproc with IoU 0.7, YOLOv5x, 25 epochs, 20% valid split, 1024 size, IOU 0.6, conf 0.1, remove (most) augs", },
    11: { "version_notes": "Remove (most) augs --- WBF preproc IoU: 0.8, version: yolov5l, epochs: 30, valid split: 20.0%, image size: 1024, detect IoU: 0.8, detect conf: 0.05", },
    # 5 Folds of the same model
    12: { "version_notes": "Augs: remove flips, mosaic @ 0.5 --- Fold: 0/5, WBF preproc IoU: 0.7, version: yolov5x, epochs: 25, valid split: 20.0%, image size: 1024, detect IoU: 0.6, detect conf: 0.1", },
    13: { "version_notes": "Augs: remove flips, mosaic @ 0.5 --- Fold: 1/5, WBF preproc IoU: 0.7, version: yolov5x, epochs: 25, valid split: 20.0%, image size: 1024, detect IoU: 0.6, detect conf: 0.1", },
    14: { "version_notes": "Augs: remove flips, mosaic @ 0.5 --- Fold: 2/5, WBF preproc IoU: 0.7, version: yolov5x, epochs: 25, valid split: 20.0%, image size: 1024, detect IoU: 0.6, detect conf: 0.1", },
    15: { "version_notes": "Augs: remove flips, mosaic @ 0.5 --- Fold: 3/5, WBF preproc IoU: 0.7, version: yolov5x, epochs: 25, valid split: 20.0%, image size: 1024, detect IoU: 0.6, detect conf: 0.1", },
    16: { "version_notes": "Augs: remove flips, mosaic @ 0.5 --- Fold: 4/5, WBF preproc IoU: 0.7, version: yolov5x, epochs: 25, valid split: 20.0%, image size: 1024, detect IoU: 0.6, detect conf: 0.1", },
    # version 17 and 18 are identical.
    # 5 Folds, WBF at 0.4
    18: { "version_notes": "Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 50; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: exp_yolov5x_50epochs_WBFat0.4iou_640image_detect0.6iou0.01conf_fold0of5", },
    19: { "version_notes": "Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 1/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_40epochs_640image_fold1of5_WBFat0.4iou_det0.6iou0.01conf", },
    20: { "version_notes": "Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 2/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_40epochs_640image_fold1of5_WBFat0.4iou_det0.6iou0.01conf", },
    21: { "version_notes": "Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 3/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_40epochs_640image_fold1of5_WBFat0.4iou_det0.6iou0.01conf", },
    22: { "version_notes": "Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 4/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_40epochs_640image_fold1of5_WBFat0.4iou_det0.6iou0.01conf", },
    #
    23: { "version_notes": "Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.65; detect conf: 0.001; exp: yolov5x_40epochs_640image_fold0of5_WBFat0.4iou_det0.65iou0.001conf", },
    24: { "version_notes": "Rad weight as number of labels, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 50; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_50epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    # 5 Folds for outliers filter with Gaussian model
    25: { "version_notes": "Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    26: { "version_notes": "Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 1/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    27: { "version_notes": "Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 2/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    28: { "version_notes": "Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 3/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    29: { "version_notes": "Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 4/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    # version 30 did very poorly in CV
    31: { "version_notes": "Filter outliers (bbox size) with quantiles, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.8; detect conf: 0.01; exp: yolov5x_outliers_quantiles_40epochs_640image_fold0of5_WBFat0.4iou_det0.8iou0.01conf", },
    32: { "version_notes": "Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 100; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gss_100epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    # versions 33-37 are versions 25-29 with TTA
    33: { "version_notes": "TTA ++ Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    34: { "version_notes": "TTA ++ Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 1/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    35: { "version_notes": "TTA ++ Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 2/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    36: { "version_notes": "TTA ++ Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 3/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    37: { "version_notes": "TTA ++ Filter outliers (bbox size) with Gaussian model, Augs: translate - 0.2, scale - 0.6, flip up - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 4/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_40epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf", },
    #
    38: { "version_notes": "Filter outliers (bbox size) with Gaussian model, use focal loss at 1.5, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 60; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gss_fl_60epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf.exp", },
    # Use GroupKFold
    39: { "version_notes": "Filter outliers (bbox size) with Gaussian model, GroupKFold, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 50; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gkf_50epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf.exp", },
    40: { "version_notes": "Filter outliers (bbox size) with Gaussian model, GroupKFold, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 1/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gkf_40epochs_640image_fold1of5_WBFat0.4iou_det0.6iou0.01conf.exp", },
    41: { "version_notes": "Filter outliers (bbox size) with Gaussian model, GroupKFold, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 2/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gkf_40epochs_640image_fold2of5_WBFat0.4iou_det0.6iou0.01conf.exp", },
    42: { "version_notes": "Filter outliers (bbox size) with Gaussian model, GroupKFold, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 3/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gkf_40epochs_640image_fold3of5_WBFat0.4iou_det0.6iou0.01conf.exp", },
    43: { "version_notes": "Filter outliers (bbox size) with Gaussian model, GroupKFold, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 4/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gkf_40epochs_640image_fold3of5_WBFat0.4iou_det0.6iou0.01conf.exp", },
    # Versions 44-48 are versions 39-43 without TTA
    44: { "version_notes": "No TTA ++ Filter outliers (bbox size) with Gaussian model, GroupKFold, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 50; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gkf_50epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf.exp", },
    45: { "version_notes": "No TTA ++ Filter outliers (bbox size) with Gaussian model, GroupKFold, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 1/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gkf_40epochs_640image_fold1of5_WBFat0.4iou_det0.6iou0.01conf.exp", },
    46: { "version_notes": "No TTA ++ Filter outliers (bbox size) with Gaussian model, GroupKFold, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 2/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gkf_40epochs_640image_fold2of5_WBFat0.4iou_det0.6iou0.01conf.exp", },
    47: { "version_notes": "No TTA ++ Filter outliers (bbox size) with Gaussian model, GroupKFold, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 3/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gkf_40epochs_640image_fold3of5_WBFat0.4iou_det0.6iou0.01conf.exp", },
    48: { "version_notes": "No TTA ++ Filter outliers (bbox size) with Gaussian model, GroupKFold, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 4/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gkf_40epochs_640image_fold3of5_WBFat0.4iou_det0.6iou0.01conf.exp", },
    #
}
# The dictionary shouldn't allow duplicate keys anyway.
assert len(set(INPUTS_YOLO_VERSIONS.keys())) == len(INPUTS_YOLO_VERSIONS), "Version dictionary has repeated keys"
for yolo_version, version_data in INPUTS_YOLO_VERSIONS.items():
    assert all(list(key_name in version_data for key_name in ["version_notes"])), f"Not all keys present in version {yolo_version} definitions"

In [3]:
#@title Select models versions { run: "auto" }

binary_version = "7: EfficientNetB3, include probabilities (for later double thresholding); image size: (640, 640); threshold: 0.8" #@param ["2: Try 1024 resolution", "3: EffNet, 1024 resolution, threshold 0.8", "4: EffNet, 1024 resolution, threshold 0.8", "5: EffNet, 512 resolution, threshold 0.8", "6: EfficientNetB3, include probabilities (for later double thresholding); image size: (1024, 1024); threshold: 0.8", "7: EfficientNetB3, include probabilities (for later double thresholding); image size: (640, 640); threshold: 0.8"]
binary_notes = binary_version.split(":")[1]
binary_version = int(binary_version.split(":")[0])


yolo_versions_include = "39, 40, 41, 42, 43" #@param {type:"string"}
yolo_versions_include = list(map(int, yolo_versions_include.split(","))) if yolo_versions_include else []
# Check that versions are distinct (no need to ensemble the same model)
assert len(yolo_versions_include) == len(set(yolo_versions_include)), "Versions to include have repeated entries"
assert all(version in INPUTS_YOLO_VERSIONS for version in yolo_versions_include), "Unknown version numbers present in version to include"
print(yolo_versions_include)

[39, 40, 41, 42, 43]


In [4]:
for yolo_version, version_data in INPUTS_YOLO_VERSIONS.items():
    version_data["path"] = f"v{yolo_version}"
    version_data["enabled"] = yolo_version in yolo_versions_include

# Filter only enabled.
INPUTS_YOLO_VERSIONS = {yolo_version: version_data for yolo_version, version_data in INPUTS_YOLO_VERSIONS.items() if version_data["enabled"]}

## Setup

In [5]:
try:
    from google.colab import drive
    drive.mount("/content/drive")
    %cd /content/drive/MyDrive/Colab\ Notebooks/kaggle
    from setup_colab import setup_colab_for_kaggle, WORK_FOLDER
    setup_colab_for_kaggle(check_env=False, local_working=True)
except:
    print("Not in Colab")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/kaggle
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Content of Drive Kaggle data dir (/content/drive/MyDrive/kaggle): ['/content/drive/MyDrive/kaggle/input', '/content/drive/MyDrive/kaggle/working', '/content/drive/MyDrive/kaggle/.ipynb_checkpoints', '/content/drive/MyDrive/kaggle/output']
Content of Kaggle data dir (/kaggle): ['/kaggle/output', '/kaggle/working', '/kaggle/input']
Content of Kaggle data subdir (/kaggle/input): ['/kaggle/input/cassava-model', '/kaggle/input/cassava-leaf-disease-classification', '/kaggle/input/googlebitemperedloss', '/kaggle/input/vbdyolo', '/kaggle/input/.ipynb_checkpoints', '/kaggle/input/vinbigdata', '/kaggle/input/vinbigdata-chest-xray-abnormalities-detection', '/kaggle/input/vinbigdata-chest-xray-origi

In [6]:
from pathlib import Path

from IPython.display import clear_output

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

INPUT_FOLDER_ORIGINAL_PNG = WORK_FOLDER / "vinbigdata-chest-xray-original-png"
INPUT_FOLDER_YOLO_OUT = WORK_FOLDER / "vbdyolo-out"
INPUT_FOLDER_BINARY = WORK_FOLDER / "vbdbinary"

for yolo_version, version_data in INPUTS_YOLO_VERSIONS.items():
    version_data["path"] = INPUT_FOLDER_YOLO_OUT / version_data["path"]

print(f"Using the following YOLO versions:")
print("\n".join(f"{yolo_version}: {version_data}" for yolo_version, version_data in INPUTS_YOLO_VERSIONS.items()))

Using the following YOLO versions:
39: {'version_notes': 'Filter outliers (bbox size) with Gaussian model, GroupKFold, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 50; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gkf_50epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf.exp', 'path': PosixPath('/kaggle/working/vbdyolo-out/v39'), 'enabled': True}
40: {'version_notes': 'Filter outliers (bbox size) with Gaussian model, GroupKFold, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 1/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gkf_40epochs_640image_fold1of5_WBFat0.4iou_det0.6iou0.01conf.exp', 'path': PosixPath('/kaggle/working/vbdyolo-out/v40'), 'enabled': True}
41: {'version_notes': 'Filter outli

## Get data from Kaggle

In [7]:
!pip install -U git+https://github.com/Witalia008/kaggle-api.git@fix-datasets-download-file-unzip
clear_output()

In [8]:
!kaggle datasets download corochann/vinbigdata-chest-xray-original-png -f test_meta.csv -p {INPUT_FOLDER_ORIGINAL_PNG} --unzip

  0% 0.00/126k [00:00<?, ?B/s]
100% 126k/126k [00:00<00:00, 39.6MB/s]


In [9]:
!pip install -U git+https://github.com/Witalia008/kaggle-api.git@add-datasets-download-version
clear_output()

In [10]:
for yolo_version, version_data in INPUTS_YOLO_VERSIONS.items():
    !rm -rf {version_data["path"]}
    !kaggle datasets download "witalia/vbdyolo-out-newest" -v {yolo_version} -p {version_data["path"]} --unzip --force

 97% 303M/311M [00:02<00:00, 105MB/s]
100% 311M/311M [00:02<00:00, 126MB/s]
 99% 308M/311M [00:01<00:00, 139MB/s]
100% 311M/311M [00:01<00:00, 163MB/s]
 96% 298M/311M [00:02<00:00, 151MB/s]
100% 311M/311M [00:02<00:00, 155MB/s]
 94% 292M/311M [00:01<00:00, 204MB/s]
100% 311M/311M [00:01<00:00, 185MB/s]
 98% 306M/311M [00:01<00:00, 198MB/s]
100% 311M/311M [00:01<00:00, 196MB/s]


In [11]:
!rm -rf {INPUT_FOLDER_BINARY}
!kaggle datasets download "witalia/vbdbinary" -v {binary_version} -p {INPUT_FOLDER_BINARY} --unzip --force

 96% 102M/106M [00:00<00:00, 152MB/s] 
100% 106M/106M [00:00<00:00, 150MB/s]


## Process YOLO output

### BBox reduction (potentially from multiple models)

In [12]:
!pip install ensemble-boxes
clear_output()

In [13]:
from ensemble_boxes import *
from typing import List

def reduce_boxes(image_labels_list: List[pd.DataFrame], iou_thr=iou_threshold, skip_box_thr=skip_box_threshold, sigma=sigma, method=None):
    label_empty = [len(df) == 0 for df in image_labels_list]

    if len(image_labels_list) == 0 or all(label_empty):
        # If there's nothing to fuse, return empty.
        return pd.DataFrame(columns=["class_id", "conf", "x_min", "y_min", "x_max", "y_max"])

    # Ensemble_boxes doesn't do well with empty inputs, so skip those.
    labels = [image_labels["class_id"].values for image_labels, empty in zip(image_labels_list, label_empty) if not empty]
    scores = [image_labels["conf"].values for image_labels, empty in zip(image_labels_list, label_empty) if not empty]
    boxes = [image_labels[["x_min", "y_min", "x_max", "y_max"]].values for image_labels, empty in zip(image_labels_list, label_empty) if not empty]

    if method == non_maximum_weighted or method == weighted_boxes_fusion:
        boxes, scores, labels = method(boxes, scores, labels, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    elif method == soft_nms:
        boxes, scores, labels = method(boxes, scores, labels, iou_thr=iou_thr, thresh=skip_box_thr, sigma=sigma)
    elif method == nms:
        boxes, scores, labels = method(boxes, scores, labels, iou_thr=iou_thr)
    else:
        if len(labels) > 1:
            raise ValueError("Cannot fuse more than 1 model without a fusing method")
        [boxes], [scores], [labels] = boxes, scores, labels

    reduced_labels = pd.DataFrame().reindex(list(range(len(labels))))
    reduced_labels["class_id"] = np.array(labels).astype(np.int32)
    reduced_labels["conf"] = scores
    reduced_labels[["x_min", "y_min", "x_max", "y_max"]] = boxes

    return reduced_labels


def reduce_max_x_per_class(image_labels: pd.DataFrame):
    max_counts = [
        ([0, 3], 3),  # Aortic Enlargement and Cardiomegaly
        ([11, 13], 25),  # Pleaural thickening and Pulmonary fibrosis
        (list(set(range(14)) - set([0, 3, 11, 13])), 50)
    ]

    result_df = image_labels

    for class_ids, max_count in max_counts:
        # Aortic Enlargement and Cardiomegaly should be max one on the image.
        one_class_mask = result_df["class_id"].isin(class_ids)

        # Take most confident boxes in each class.
        result_df_one_class = result_df[one_class_mask].sort_values("conf", ascending=False).groupby("class_id").head(max_count)

        result_df = result_df[~one_class_mask]
        result_df = result_df.append(result_df_one_class)

    return result_df


def reduce_by_location_boundaries(image_labels: pd.DataFrame):
    image_labels["x_mid"] = (image_labels["x_min"] + image_labels["x_max"]) / 2
    image_labels["y_mid"] = (image_labels["y_min"] + image_labels["y_max"]) / 2

    valid_labels_mask = image_labels["class_id"] == image_labels["class_id"]

    # Remove those that are in wrong location for Aortic Enlargement
    aort_enlarg_wrong_loc = (image_labels["class_id"] == 0) & \
        ((image_labels["x_mid"] < 0.3) | (image_labels["x_mid"] > 0.8) | (image_labels["y_mid"] > 0.6))
    valid_labels_mask = valid_labels_mask & (~aort_enlarg_wrong_loc)

    # Remove those that are in wrong location for 
    cardiomeg_wrong_loc = (image_labels["class_id"] == 3) & \
        ((image_labels["x_mid"] < 0.4) | (image_labels["x_mid"] > 0.9) | (image_labels["y_mid"] < 0.4))
    valid_labels_mask = valid_labels_mask & (~cardiomeg_wrong_loc)

    # Any bbox shouldn't be very close to border (except for "Other lesion")
    any_bbox_wrong_loc = (image_labels["class_id"] != 9) & \
        ((image_labels["x_mid"] < 0.03) | (image_labels["x_mid"] > 0.97) | \
            (image_labels["y_mid"] < 0.1) | (image_labels["y_mid"] > 0.9))
    valid_labels_mask = valid_labels_mask & (~any_bbox_wrong_loc)

    # Remove the temporary columns.
    image_labels = image_labels.drop(columns=["x_mid", "y_mid"])

    return image_labels[valid_labels_mask]

### Read and transform labels

In [14]:
def read_prediction_labels(filename: Path):
    yolo_columns = ["class_id", "x_centre", "y_centre", "bw", "bh", "conf"]
    if filename.exists():
        labels: pd.DataFrame = pd.read_csv(filename, delimiter=" ", header=None)
        labels.columns = yolo_columns
    else:
        labels: pd.DataFrame = pd.DataFrame(columns=yolo_columns)

    # Convert YOLO format (x_centre, y_centre, bw, bh) to competition format (x_min, y_min, x_max, y_max)
    labels["x_min"] = (labels["x_centre"] - labels["bw"] / 2).clip(0, 1)
    labels["y_min"] = (labels["y_centre"] - labels["bh"] / 2).clip(0, 1)
    labels["x_max"] = (labels["x_centre"] + labels["bw"] / 2).clip(0, 1)
    labels["y_max"] = (labels["y_centre"] + labels["bh"] / 2).clip(0, 1)
    labels = labels.drop(columns=["x_centre", "y_centre", "bw", "bh"])
    # After dropping, conf column should become the second one.
    assert labels.columns.to_list() == ["class_id", "conf", "x_min", "y_min", "x_max", "y_max"], "Unexpected columns found"

    return labels


def scale_labels(labels: pd.DataFrame, image_w: int, image_h: int):
    # Scale coordinates to image's size. Clip to make sure it's not out of bounds of the image.
    labels[["x_min", "x_max"]] = (labels[["x_min", "x_max"]] * image_w).round().astype(np.int32).clip(0, image_w - 1)
    labels[["y_min", "y_max"]] = (labels[["y_min", "y_max"]] * image_h).round().astype(np.int32).clip(0, image_h - 1)

    return labels


def labels_to_string(labels: pd.DataFrame):
    # Empty DataFrame means there was no YOLO output file or "reduce" removed all bboxes.
    # Means "No finding"
    if len(labels) == 0:
        return "14 1 0 0 1 1"
    assert labels.columns.to_list() == ["class_id", "conf", "x_min", "y_min", "x_max", "y_max"], "Unexpected columns found"
    # Convert all rows to one prediction string
    return " ".join(labels.to_string(header=False, index=False).split())

### Process YOLO output files, and calculate the predictions

In [15]:
import ensemble_boxes

results_yolo_df = pd.DataFrame(columns=["image_id", "PredictionString"])
results_yolo_full_df = pd.DataFrame(columns=["image_id", "class_id", "conf", "x_min", "y_min", "x_max", "y_max"])

test_metadata = pd.read_csv(INPUT_FOLDER_ORIGINAL_PNG / "test_meta.csv")
test_metadata = test_metadata.set_index("image_id").to_dict("index")

for image_id, image_dims in tqdm(test_metadata.items(), total=len(test_metadata)):
    # Read YOLO output to pandas DataFrame.
    prediction_labels = [read_prediction_labels(
        version_data["path"] / "labels_pred" / f"{image_id}.txt"
    ) for version_data in INPUTS_YOLO_VERSIONS.values()]

    # Reduce bboxes.
    method = None if reduce_method == "None" else getattr(ensemble_boxes, reduce_method)
    prediction_labels = reduce_boxes(prediction_labels, method=method)

    if reduce_filter:
        # Reduce those that are in anomalous locations.
        prediction_labels = reduce_by_location_boundaries(prediction_labels)

        # Reduce those that should only have one class
        prediction_labels = reduce_max_x_per_class(prediction_labels)

    # Append all the boxes to full df for later analysis.
    prediction_labels_full = prediction_labels.copy()
    prediction_labels_full["image_id"] = image_id
    results_yolo_full_df = results_yolo_full_df.append(prediction_labels_full)

    # Scale them to the original size.
    # NOTE: dim0 and dim1 are reversed: y-axis, x-axis!
    prediction_labels = scale_labels(prediction_labels, image_dims["dim1"], image_dims["dim0"])

    # Convert resulting bboxes from DataFrame to string.
    prediction_str = labels_to_string(prediction_labels)

    results_yolo_df = results_yolo_df.append({"image_id": image_id, "PredictionString": prediction_str}, ignore_index=True)

results_yolo_df.sample(10, random_state=10)

,image_id,PredictionString
1779,9b41c13ac918adcc1261f750a1d61b40,11 0.395355 673 457 865 539 8 0.373157 426 175...
341,202dc97c4b3ec7e50f4813456938500a,3 0.909375 704 1227 1555 1524 0 0.738407 960 6...
1276,711dc542c6c7e2b21889838cd050a810,7 0.301636 1352 986 1725 1514 7 0.182586 459 1...
1012,5ba0a669fd3430e9386758e838210b98,0 0.361715 1782 1181 2073 1479 7 0.301018 894 ...
470,2b2810284313c5547241b38668f67b2c,0 0.856738 1076 962 1356 1285 3 0.302486 750 1...
1755,990b0dcd83353894c4a2e73738cd6c25,3 0.326477 1038 1208 2146 1664 5 0.190576 1622...
2773,ec6c4e81b448275577d3e4a74acbe443,0 0.851074 1094 690 1317 933 3 0.574756 846 13...
1085,6100bc75e0a668c382d92d5eb9038076,3 0.801856 666 1097 1548 1392 0 0.236011 1024 ...
2055,b1a596318e7bfdcbb74044bf6d38c6ed,3 0.669238 637 1245 1455 1552 0 0.621713 959 5...
2585,de7aa2b6adfb9db39489c019f5f3a1a1,3 0.893750 680 1376 1799 1736 0 0.877051 1036 ...


## Merge with Binary classifier output

In [16]:
results_binary_df = pd.read_csv(INPUT_FOLDER_BINARY / "prediction.csv")
display(results_binary_df.head())

results_df = results_yolo_df.merge(results_binary_df, on="image_id")

if "confidence" in results_df.columns:
    # If confidence of "No finding" is very high, replace with "No finding"
    results_df.loc[results_df["confidence"] >= binary_normal_threshold, "PredictionString"] = "14 1 0 0 1 1"
    # If confidence of "No finding" is medium, add "No finding" with confidence to the object detector's bboxes.
    # Skip those that are already "No finding".
    results_df.loc[
            (results_df["confidence"] < binary_normal_threshold) & \
            (results_df["confidence"] > binary_abnormal_threshold) & \
            (~(results_df["PredictionString"] == "14 1 0 0 1 1")),
        "PredictionString"] += results_df["confidence"].apply(lambda prob: f" 14 {prob:.6f} 0 0 1 1")
    # If confidence of "No finding" is low, i.e. confidence of abnormality is high, do nothing.
else:
    # Just use binary classifier's suggestion as to which to set as "No finding"
    results_df.loc[results_df["class_name"] == "normal", "PredictionString"] = "14 1 0 0 1 1"

results_df = results_df[["image_id", "PredictionString"]]

results_df.to_csv(WORK_FOLDER / "submission.csv", index=False)
display(results_df.head())

,image_id,class_name,confidence
0,980141b50ada8a624c9a1c34c91c95d8,abnormal,0.470578
1,f362fd880b0149646ccbb760be5f6354,normal,0.994644
2,0291515f5d14c34180a15712a55bf7bd,abnormal,0.089130
3,6cf8916303a07f3909297be170f21f7a,normal,0.970541
4,c6829051ecb281656c987fa2cfe5c706,abnormal,0.167964


,image_id,PredictionString
0,002a34c58c5b758217ed1f584ccbcfe9,14 1 0 0 1 1
1,004f33259ee4aef671c2b95d54e4be68,0 0.643468 1264 586 1534 897 0 0.144057 1098 5...
2,008bdde2af2462e86fd373a445d0f4cd,3 0.884570 1094 1402 1938 1785 0 0.869629 1425...
3,009bc039326338823ca3aa84381f17f1,3 0.857715 663 1053 1555 1354 0 0.605853 993 4...
4,00a2145de1886cb9eb88869c85d74080,3 0.889063 776 1281 1861 1638 0 0.827246 1125 ...


In [17]:
images_no_finding = results_df[results_df["PredictionString"] == "14 1 0 0 1 1"].image_id.values
results_yolo_full_df = results_yolo_full_df[~results_yolo_full_df.image_id.isin(images_no_finding)]

## Submission overview

Overall count of labels

In [18]:
display(results_df["PredictionString"].apply(lambda s: int(len(s.split(" ")) / 6)).sum())

130449

Counts of each class per image

In [19]:
class_counts_per_label = results_yolo_full_df.groupby("image_id").class_id.value_counts().unstack(level=-1).reset_index().fillna(0)
class_counts_per_label.index.rename("index")
class_counts_per_label.head(2)

class_id,image_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,004f33259ee4aef671c2b95d54e4be68,3.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,3.0,2.0,1.0,4.0,0.0,0.0
1,008bdde2af2462e86fd373a445d0f4cd,2.0,0.0,2.0,1.0,0.0,0.0,0.0,3.0,1.0,1.0,0.0,13.0,0.0,4.0


Average counts

In [20]:
class_counts_per_label.describe()

class_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13
count,1839.000000,1839.000000,1839.000000,1839.000000,1839.000000,1839.000000,1839.000000,1839.000000,1839.000000,1839.000000,1839.000000,1839.000000,1839.000000,1839.000000
mean,2.676998,0.280587,2.601414,1.064709,0.511691,3.954867,1.505710,4.839043,7.852094,8.513866,3.324089,17.510060,0.523654,14.205003
std,2.776018,1.241120,6.219631,0.894092,2.066561,6.381729,3.683159,7.662081,24.191449,16.369765,6.978236,12.372386,2.366883,20.562965
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,8.000000,0.000000,2.000000
50%,2.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,2.000000,3.000000,4.000000,1.000000,16.000000,0.000000,6.000000
75%,3.000000,0.000000,2.000000,1.000000,0.000000,5.000000,1.000000,6.000000,7.000000,9.000000,3.000000,24.000000,0.000000,17.000000
max,26.000000,15.000000,97.000000,9.000000,29.000000,56.000000,47.000000,79.000000,520.000000,316.000000,53.000000,90.000000,53.000000,175.000000


Stats of bboxes per class

In [21]:
class_bbox_locations = results_yolo_full_df.copy()
class_bbox_locations["x_mid"] = (class_bbox_locations["x_min"] + class_bbox_locations["x_max"]) / 2
class_bbox_locations["y_mid"] = (class_bbox_locations["y_min"] + class_bbox_locations["y_max"]) / 2
class_bbox_locations = class_bbox_locations[["class_id", "x_mid", "y_mid", "conf"]]
class_bbox_locations.groupby("class_id").describe()

x_mid                      ...      conf                    
            count      mean       std  ...       50%       75%       max
class_id                               ...                              
0          4923.0  0.554542  0.063202  ...  0.019261  0.256471  0.898535
1           516.0  0.603504  0.219928  ...  0.009290  0.030876  0.422559
2          4784.0  0.518155  0.203666  ...  0.007505  0.023071  0.707422
3          1958.0  0.570610  0.059835  ...  0.225879  0.721338  0.923535
4           941.0  0.479488  0.225534  ...  0.012024  0.047662  0.479102
5          7273.0  0.505376  0.254976  ...  0.008926  0.031250  0.616894
6          2769.0  0.486238  0.240673  ...  0.009540  0.035852  0.462110
7          8899.0  0.480325  0.234231  ...  0.009167  0.031049  0.583496
8         14440.0  0.479038  0.244979  ...  0.005833  0.016671  0.774219
9         15657.0  0.467629  0.245584  ...  0.005655  0.015302  0.691064
10         6113.0  0.455638  0.338732  ...  0.008975  0.031763  0.774317
11        32201.0  0.500713  0.211791  ...  0.008643  0.026781  0.645854
12          963.0  0.462784  0.260187  ...  0.006262  0.016513  0.711660
13        26123.0  0.465629  0.236069  ...  0.006598  0.019946  0.660075

[14 rows x 24 columns]

Total number of "No finding" before binary classifier filter

In [22]:
print((results_yolo_df["PredictionString"] == "14 1 0 0 1 1").sum())

0


Total number of "No finding"

In [23]:
(results_df["PredictionString"] == "14 1 0 0 1 1").sum()

1161

Total with class "No finding" appended

In [24]:
print(results_df["PredictionString"].str.contains(r" 14 \d\.\d+ 0 0 1 1").sum())

1728


## Submit to Kaggle

In [25]:
submission_message = " ***** ".join([
    version_notes,
    "; ".join([
        f"Reduce method: {reduce_method}",
        f"IoU {iou_threshold}",
        f"Conf {skip_box_threshold}",
        f"Sigma {sigma}",
        f"Binary Normal Thresh: {binary_normal_threshold}",
        f"Binary Abnormal Thresh: {binary_abnormal_threshold}",
    ]),
    f"Binary CLF: {binary_version} -{binary_notes}",
    "Ensemble YOLO: " + "; ".join(f"{yolo_version} - {version_data['version_notes']}" for yolo_version, version_data in INPUTS_YOLO_VERSIONS.items()),
])
!echo "{submission_message}"

if SUBMIT:
    !kaggle competitions submit \
        vinbigdata-chest-xray-abnormalities-detection \
        -f {WORK_FOLDER}/submission.csv \
        -m "{submission_message}"
    !sleep 10
    !kaggle competitions submissions vinbigdata-chest-xray-abnormalities-detection

BBox reduction ***** Reduce method: weighted_boxes_fusion; IoU 0.6; Conf 0.01; Sigma 0.1; Binary Normal Thresh: 0.95; Binary Abnormal Thresh: 0.05 ***** Binary CLF: 7 - EfficientNetB3, include probabilities (for later double thresholding); image size ***** Ensemble YOLO: 39 - Filter outliers (bbox size) with Gaussian model, GroupKFold, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 0/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 50; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gkf_50epochs_640image_fold0of5_WBFat0.4iou_det0.6iou0.01conf.exp; 40 - Filter outliers (bbox size) with Gaussian model, GroupKFold, Augs: translate - 0.2, scale - 0.6, flip ud - 0, flip lr - 0.5, mosaic - 1.0 --- Fold: 1/5; WBF preproc IoU: 0.4; version: yolov5x; epochs: 40; valid split: 20.0%; image size: 640; detect IoU: 0.6; detect conf: 0.01; exp: yolov5x_outliers_tta_gkf_40epochs_640image_fold1of5_WBFat0.4iou_d